In [1]:
import warnings
warnings.filterwarnings('ignore')

# Collecting pairs of person photos

In [2]:
import os 
import pandas as pd

In [ ]:
path = 'dataset/lfw_funneled/'

In [ ]:
pairs = ''
with open(f'{path}pairs.txt', 'r') as file:
    pairs = file.read()

In [ ]:
df = pd.DataFrame(list(map(lambda x: x.split('\t'), pairs.split('\n')[1:301])), columns=['name', 'pic1', 'pic2'])
df.head()

In [ ]:
for i in [1, 2]:
    df[f'pic{i}'] = df[f'pic{i}'].apply(lambda x: x.zfill(4))
    df[f'path{i}'] = str(path) + df['name'] + '/' + df['name'] + '_' + df[f'pic{i}'] + '.jpg'
df.head()

In [ ]:
df.to_csv('pairs.csv')

# Adding scaled with Super-Resolution pictures

In [29]:
df = pd.read_csv('pairs.csv', index_col=0)
df.head()

,name,pic1,pic2,path1,path2,recognized,path1_x2,path2_x2
0,Abel_Pacheco,1,4,dataset/lfw_funneled/Abel_Pacheco/Abel_Pacheco...,dataset/lfw_funneled/Abel_Pacheco/Abel_Pacheco...,True,output/Abel_Pacheco_0001.jpg_result.jpg,output/Abel_Pacheco_0004.jpg_result.jpg
1,Akhmed_Zakayev,1,3,dataset/lfw_funneled/Akhmed_Zakayev/Akhmed_Zak...,dataset/lfw_funneled/Akhmed_Zakayev/Akhmed_Zak...,True,output/Akhmed_Zakayev_0001.jpg_result.jpg,output/Akhmed_Zakayev_0003.jpg_result.jpg
2,Akhmed_Zakayev,2,3,dataset/lfw_funneled/Akhmed_Zakayev/Akhmed_Zak...,dataset/lfw_funneled/Akhmed_Zakayev/Akhmed_Zak...,True,output/Akhmed_Zakayev_0002.jpg_result.jpg,output/Akhmed_Zakayev_0003.jpg_result.jpg
3,Amber_Tamblyn,1,2,dataset/lfw_funneled/Amber_Tamblyn/Amber_Tambl...,dataset/lfw_funneled/Amber_Tamblyn/Amber_Tambl...,True,output/Amber_Tamblyn_0001.jpg_result.jpg,output/Amber_Tamblyn_0002.jpg_result.jpg
4,Anders_Fogh_Rasmussen,1,3,dataset/lfw_funneled/Anders_Fogh_Rasmussen/And...,dataset/lfw_funneled/Anders_Fogh_Rasmussen/And...,True,output/Anders_Fogh_Rasmussen_0001.jpg_result.jpg,output/Anders_Fogh_Rasmussen_0003.jpg_result.jpg


In [5]:
pics_to_scale = set(df['path1'].values) | set(df['path2'].values)

In [ ]:
%%time
for pic in pics_to_scale[:]:
    abc = f'--file={pic}'
    %run sr.py {abc}

In [30]:
for i in [1, 2]:
    df[f'path{i}_x2'] = 'dataset/output/' + df[f'path{i}'].apply(lambda x: x.split('/')[-1][:-4]) + '_result.jpg'

In [31]:
df.to_csv('pairs.csv')

# Pairs recognition

In [7]:
from recognizer import Recognizer
from skimage.io import imread

THRESHOLD = 0.9

In [32]:
df = pd.read_csv('pairs.csv', index_col=0)
df.head()

,name,pic1,pic2,path1,path2,recognized,path1_x2,path2_x2
0,Abel_Pacheco,1,4,dataset/lfw_funneled/Abel_Pacheco/Abel_Pacheco...,dataset/lfw_funneled/Abel_Pacheco/Abel_Pacheco...,True,output/Abel_Pacheco_0001_result.jpg,output/Abel_Pacheco_0004_result.jpg
1,Akhmed_Zakayev,1,3,dataset/lfw_funneled/Akhmed_Zakayev/Akhmed_Zak...,dataset/lfw_funneled/Akhmed_Zakayev/Akhmed_Zak...,True,output/Akhmed_Zakayev_0001_result.jpg,output/Akhmed_Zakayev_0003_result.jpg
2,Akhmed_Zakayev,2,3,dataset/lfw_funneled/Akhmed_Zakayev/Akhmed_Zak...,dataset/lfw_funneled/Akhmed_Zakayev/Akhmed_Zak...,True,output/Akhmed_Zakayev_0002_result.jpg,output/Akhmed_Zakayev_0003_result.jpg
3,Amber_Tamblyn,1,2,dataset/lfw_funneled/Amber_Tamblyn/Amber_Tambl...,dataset/lfw_funneled/Amber_Tamblyn/Amber_Tambl...,True,output/Amber_Tamblyn_0001_result.jpg,output/Amber_Tamblyn_0002_result.jpg
4,Anders_Fogh_Rasmussen,1,3,dataset/lfw_funneled/Anders_Fogh_Rasmussen/And...,dataset/lfw_funneled/Anders_Fogh_Rasmussen/And...,True,output/Anders_Fogh_Rasmussen_0001_result.jpg,output/Anders_Fogh_Rasmussen_0003_result.jpg


In [9]:
recognizer = Recognizer()

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Restoring parameters from ./facenet_models/facenet/model-20180408-102900.ckpt-90


In [14]:
def recongize(row):
    photo1 = imread(row['path1'])
    photo2 = imread(row['path2'])
    similarity = recognizer.get_best_similarity(photo1, photo2)
    return similarity < THRESHOLD

In [15]:
%%time
df['recognized'] = df[['path1', 'path2']].apply(recongize, axis=1)

Wall time: 1min 27s


In [33]:
def recongize_x2(row):
    photo1 = imread(row['path1_x2'])
    photo2 = imread(row['path2_x2'])
    similarity = recognizer.get_best_similarity(photo1, photo2)
    return similarity < THRESHOLD

In [34]:
%%time
df['recognized_x2'] = df[['path1_x2', 'path2_x2']].apply(recongize_x2, axis=1)

Wall time: 3min 17s


In [35]:
df.head()

,name,pic1,pic2,path1,path2,recognized,path1_x2,path2_x2,recognized_x2
0,Abel_Pacheco,1,4,dataset/lfw_funneled/Abel_Pacheco/Abel_Pacheco...,dataset/lfw_funneled/Abel_Pacheco/Abel_Pacheco...,True,output/Abel_Pacheco_0001_result.jpg,output/Abel_Pacheco_0004_result.jpg,True
1,Akhmed_Zakayev,1,3,dataset/lfw_funneled/Akhmed_Zakayev/Akhmed_Zak...,dataset/lfw_funneled/Akhmed_Zakayev/Akhmed_Zak...,True,output/Akhmed_Zakayev_0001_result.jpg,output/Akhmed_Zakayev_0003_result.jpg,True
2,Akhmed_Zakayev,2,3,dataset/lfw_funneled/Akhmed_Zakayev/Akhmed_Zak...,dataset/lfw_funneled/Akhmed_Zakayev/Akhmed_Zak...,True,output/Akhmed_Zakayev_0002_result.jpg,output/Akhmed_Zakayev_0003_result.jpg,True
3,Amber_Tamblyn,1,2,dataset/lfw_funneled/Amber_Tamblyn/Amber_Tambl...,dataset/lfw_funneled/Amber_Tamblyn/Amber_Tambl...,True,output/Amber_Tamblyn_0001_result.jpg,output/Amber_Tamblyn_0002_result.jpg,True
4,Anders_Fogh_Rasmussen,1,3,dataset/lfw_funneled/Anders_Fogh_Rasmussen/And...,dataset/lfw_funneled/Anders_Fogh_Rasmussen/And...,True,output/Anders_Fogh_Rasmussen_0001_result.jpg,output/Anders_Fogh_Rasmussen_0003_result.jpg,True


In [36]:
df['recognized'].value_counts()

True     246
False     54
Name: recognized, dtype: int64

In [37]:
df['recognized_x2'].value_counts()

True     253
False     47
Name: recognized_x2, dtype: int64

In [38]:
df.to_csv('pairs.csv')

# Evaluating results

In [3]:
df = pd.read_csv('pairs.csv', index_col=0)
df.head()

,name,pic1,pic2,path1,path2,recognized,path1_x2,path2_x2,recognized_x2
0,Abel_Pacheco,1,4,dataset/lfw_funneled/Abel_Pacheco/Abel_Pacheco...,dataset/lfw_funneled/Abel_Pacheco/Abel_Pacheco...,True,output/Abel_Pacheco_0001_result.jpg,output/Abel_Pacheco_0004_result.jpg,True
1,Akhmed_Zakayev,1,3,dataset/lfw_funneled/Akhmed_Zakayev/Akhmed_Zak...,dataset/lfw_funneled/Akhmed_Zakayev/Akhmed_Zak...,True,output/Akhmed_Zakayev_0001_result.jpg,output/Akhmed_Zakayev_0003_result.jpg,True
2,Akhmed_Zakayev,2,3,dataset/lfw_funneled/Akhmed_Zakayev/Akhmed_Zak...,dataset/lfw_funneled/Akhmed_Zakayev/Akhmed_Zak...,True,output/Akhmed_Zakayev_0002_result.jpg,output/Akhmed_Zakayev_0003_result.jpg,True
3,Amber_Tamblyn,1,2,dataset/lfw_funneled/Amber_Tamblyn/Amber_Tambl...,dataset/lfw_funneled/Amber_Tamblyn/Amber_Tambl...,True,output/Amber_Tamblyn_0001_result.jpg,output/Amber_Tamblyn_0002_result.jpg,True
4,Anders_Fogh_Rasmussen,1,3,dataset/lfw_funneled/Anders_Fogh_Rasmussen/And...,dataset/lfw_funneled/Anders_Fogh_Rasmussen/And...,True,output/Anders_Fogh_Rasmussen_0001_result.jpg,output/Anders_Fogh_Rasmussen_0003_result.jpg,True


In [8]:
df[(df['recognized'] == True) & (df['recognized_x2'] == True)].shape[0]

235

In [9]:
df[(df['recognized'] == False) & (df['recognized_x2'] == True)].shape[0]

18

In [10]:
df[(df['recognized'] == True) & (df['recognized_x2'] == False)].shape[0]

11

In [11]:
df[(df['recognized'] == False) & (df['recognized_x2'] == False)].shape[0]

36